In [ ]:
# default_exp play01


In [ ]:
#export
# spread jax work over 4 virtual cpu cores:
import os
import multiprocessing
cpu_count=multiprocessing.cpu_count()
print("jax will spread work to {} cpus".format(cpu_count))
os.environ["XLA_FLAGS"]="--xla_force_host_platform_device_count={}".format(cpu_count)


In [ ]:
#export
import time
import numpy as np
import pandas as pd
import jax
import gr
import argparse
import jax.numpy as jnp
import jax.config
import jax.scipy.optimize
import jax.experimental.maps
import jax.numpy.linalg
import jax.nn
import numpy.random
from gr.pygr import mlab
from jax.experimental.maps import xmap


In [ ]:
#export
jax.config.update("jax_enable_x64", True)
_code_git_version="bdcbc4696db16e38cefae324f6b415cd15b111ab"
_code_repository="https://github.com/plops/cl-py-generator/tree/master//example/75_jax_render/source/01_play.ipynb"
_code_generation_time="23:36:41 of Thursday, 2024-05-09 (GMT+1)"
start_time=time.time()
debug=True


In [ ]:
#export
df_status=pd.DataFrame([dict(name="numpy", version=np.__version__), dict(name="pandas", version=pd.__version__), dict(name="jax", version=jax.__version__), dict(name="gr", version=gr.__version__)])
print(df_status)


In [ ]:
def volumetric_rendering(color, sigma, z_vals, dirs):
    # formulate rendering process for semi-transparent volumes that mimic front-to-back additive blending
# as camera ray traverses volume of inhomogeneous material, it accumulates color in proportion to the local color and density of the material at each point along its path
    eps=(1.00e-10)
    inv_eps=(((1.0    ))/(eps))
    z_right=z_vals[...,1:]
    z_left=z_vals[...,:-1]
    dists=((z_right)-(z_left))
    # fraction of light stuck in each voxel
    alpha=(((1.0    ))-(jnp.exp(((-1)*(jax.nn.relu(sigma))*(dists)))))
    accum_prod=jnp.concatenate([jnp.ones_like(alpha[...,:1], alpha.dtype), jnp.cumprod((((-1.0    ))+(alpha[...,:-1])+(eps)), axis=-1)])
    # absolute amount of light stuck in each voxel
    weights=((alpha)*(accum_prod))
    comp_color=((weights[...,None])*(jax.nn.sigmoid(color))).sum(axis=-2)
    # weighted average of the depths by contribution to final color
    depth=((weights)*(z_left)).sum(axis=-1)
    # total amount of light absorbed along the ray
    acc=weights.sum(axis=-1)
    # equivalent to disp = 1/max(eps, where(acc>eps,depth/acc,0))
# but more efficient and stable
# to model occlusions the ray accumulates not only color but also opacity
# if accumulated opacity reaches 1 for example when the ray traverses an opaque region then no further color can be accumulated on the ray (not sure if this code behaves like this)
    disparity=((acc)/(depth))
    disparity=jnp.where(((((0)<(disparity))) & (((disparity)<(inv_eps))) & (((eps)<(acc)))), disparity, inv_eps)
    return comp_color
